In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import seaborn as sns
from random import randrange,uniform
from sklearn.model_selection import GridSearchCV
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.decomposition import PCA
# Input data files are available in the "../input/" directory.
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from sklearn.utils import shuffle
from sklearn.ensemble import BaggingClassifier

In [ ]:
os.chdir("C:/Users/hp/Desktop/DATA SCIENTISTS")

In [ ]:
#Load the data
santander_train = pd.read_csv("train.csv")
santander_test = pd.read_csv("test.csv")

In [ ]:
santander_test.head()

In [ ]:
#Outlier analysis
import seaborn as sns
x_train=santander_train.iloc[:,2:]
y_train=santander_train['target']
sns.boxplot(x=santander_train['var_2'])

In [ ]:
#save numeric names
cnames= santander_train.iloc[:, 1:201]
#Detect and delete outliers from data
for i in cnames:
    print(i)
    q75, q25 = np.percentile(santander_train.loc[:,i], [75 ,25])
    iqr = q75 - q25
    min = q25 - (iqr*1.5)
    max = q75 + (iqr*1.5)
    print(min)
    print(max)
    santander_train = santander_train.drop(santander_train[santander_train.loc[:,i] < min].index)
    santander_train = santander_train.drop(santander_train[santander_train.loc[:,i] > max].index)

In [ ]:
santander_train.shape

In [ ]:
#Detect and replace with NA
#Extract quartiles
q75, q25 = np.percentile(santander_train['var_44'], [75 ,25])
#Calculate IQR
iqr = q75 - q25
#Calculate inner and outer fence
minimum = q25 - (iqr*1.5)
maximum = q75 + (iqr*1.5)
#Replace with NA
santander_train.loc[santander_train['var_44'] < minimum,:'var_44'] = np.nan
santander_train.loc[santander_train['var_44'] > maximum,:'var_44'] = np.nan

In [ ]:
#Missing value Analysis
#Create dataframe with missing percentage
missing_val = pd.DataFrame(santander_train.isnull().sum())
#Reset index
missing_val = missing_val.reset_index()
#Rename variable
missing_val = missing_val.rename(columns = {'index': 'Variables', 0: 'Missing_percentage'})
#Calculate percentage
missing_val['Missing_percentage'] = (missing_val['Missing_percentage']/len(santander_train))*100
#descending order
missing_val = missing_val.sort_values('Missing_percentage', ascending = False).reset_index(drop = True)

In [ ]:
#imputation method
#actual = 11.7058
#Mean = 10.628882
#Median =  10.4940
#KNN = 10.5792
#create missing value
santander_train['var_0'].loc[4] = np.nan

In [ ]:
#calculate using mean
santander_train['var_0'] = santander_train['var_0'].fillna(santander_train['var_0'].mean())

In [ ]:
# calculate using median
santander_train['var_0'] = santander_train['var_0'].fillna(santander_train['var_0'].median())

In [ ]:
#KNN imputation
#Assigning levels to the categories
lis = []
for i in range(0, santander_train.shape[1]):
    print(i)
    if(santander_train.iloc[:,i].dtypes == 'object'):
        santander_train.iloc[:,i] = pd.Categorical(santander_train.iloc[:,i])
        santander_train.iloc[:,i] = santander_train.iloc[:,i].cat.codes 
        santander_train.iloc[:,i] = santander_train.iloc[:,i].astype('object')
        lis.append(santander_train.columns[i])

In [ ]:
#Apply KNN imputation algorithm\n",
santander_train = pd.DataFrame(kNN(k = 3).fit_transform(santander_train), columns = santander_train.columns)

In [ ]:
#Freeze imputation using median
santander_train = santander_train.fillna(santander_train.median())

In [ ]:
#Convert into proper datatypes
for i in lis:
    santander_train.loc[:,i] = santander_train.loc[:,i].astype('object')

In [ ]:
type(santander_train)

In [ ]:
santander_train.describe

In [ ]:
#Feature selection
##Correlation analysis
#Correlation plot
cnames= santander_train.iloc[:, 3:201]
df_corr= santander_train.iloc[:,3:201]

In [ ]:
#Set the width and hieght of the plot
f, ax = plt.subplots(figsize=(9, 9))
#Generate correlation matrix
corr = df_corr.corr()
#Plot using seaborn library
f, ax = plt.subplots(figsize=(9, 9))
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)

In [ ]:
santander_train = santander_train.drop(['var_25', 'var_36', 'var_55', 'var_79', 'var_106', 'var_119', 'var_162', 'var_183'], axis=1)

In [ ]:
santander_train.columns

In [ ]:
#Feature Scaling
df = santander_train.copy()
santander_train = df.copy()

In [ ]:
#Normality check\n",
%matplotlib inline 
plt.hist(santander_train['var_100'], bins='auto')

In [ ]:
#Normalisation
cnames= santander_train.iloc[:, 3:201]
for i in cnames:
    print(i)
    santander_train[i] = (santander_train[i] - min(santander_train[i]))/(max(santander_train[i]) - min(santander_train[i]))

In [ ]:
#Divide data into train and test\n",
X = santander_train.values[:, 2:193]
Y = santander_train.values[:,1]
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.2)

In [ ]:
#Let us prepare data for logistic regression\n",
#replace target categories with Yes or No\n",
santander_train['target'] = santander_train['target'].replace('No', 0)
santander_train['target'] = santander_train['target'].replace('Yes', 1)

In [ ]:
#Create logistic data. Save target variable first
santander_train_logit = pd.DataFrame(santander_train['target'])

In [ ]:
santander_train_logit = santander_train_logit.join(santander_train['cnames'])

In [ ]:
santander_train_logit.head(20)

In [ ]:
Sample_Index = np.random.rand(len(santander_train_logit)) < 0.8

In [ ]:
train = santander_train_logit[Sample_Index]
test = santander_train_logit[~Sample_Index]

In [ ]:
#select column indexes for independent variables
train_cols = train.columns[3:300]

In [ ]:
#Built Logistic Regression
import statsmodels.api as sm
logit = sm.Logit(train['target'], train[train_cols]).fit()
logit.summary()

In [ ]:
#Predict test data
test['Actual_prob'] = logit.predict(test[train_cols])
test['ActualVal'] = 1
test.loc[test.Actual_prob < 0.5, 'ActualVal'] = 0

In [ ]:
#Build confusion matrix
CM = pd.crosstab(test['target'], test['ActualVal'])
#let us save TP, TN, FP, FN
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]
#check accuracy of model
accuracy_score(y_test, y_pred)*100
((TP+TN)*100)/(TP+TN+FP+FN)
(FN*100)/(FN+TP)
#Accuracy: 83.33
#FNR: 52.89

In [ ]:
#KNN implementation
from sklearn.neighbors import KNeighborsClassifier
KNN_model = KNeighborsClassifier(n_neighbors = 9).fit(X_train, y_train)

In [ ]:
#predict test cases
KNN_Predictions = KNN_model.predict(X_test)

In [ ]:
#build confusion matrix
CM = pd.crosstab(y_test, KNN_Predictions)
#let us save TP, TN, FP, FN
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]
#check accuracy of model
accuracy_score(y_test, y_pred)*100
((TP+TN)*100)/(TP+TN+FP+FN)
#False Negative rate
(FN*100)/(FN+TP)
#Accuracy: 92.23
#FNR: 81.37

In [ ]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
#Naive Bayes implementation\n",
NB_model = GaussianNB().fit(X_train, y_train)

In [ ]:
#predict test cases
NB_Predictions = NB_model.predict(X_test)

In [ ]:
#Build confusion matrix
CM = pd.crosstab(y_test, NB_Predictions)
#let us save TP, TN, FP, FN
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]
#check accuracy of model
accuracy_score(y_test, y_pred)*100
((TP+TN)*100)/(TP+TN+FP+FN)
#False Negative rate
(FN*100)/(FN+TP)
#Accuracy: 86.92
#FNR: 53.44